# Maersk Triangulation

## Setup

### Imports & API

In [2]:
import pandas as pd
import requests
from datetime import datetime, timedelta
import requests
import concurrent.futures


## Load Files

In [2]:
# Load one of the Excel files
fro_worklist_path = "C://Users//Ani//Downloads//FRO worklist dump - CMU.xlsx"
fro_list_path = 'C://Users//Ani//Downloads//FRO list - CMU.xlsx'
geocodes_list_path = 'C://Users//Ani//Downloads//Geocodes list - CMU.xlsx'
location_list_path = 'C://Users//Ani//Downloads//Location list - CMU.xlsx'

# Read the data from the Excel file
fro_worklist_data = pd.read_excel(fro_worklist_path)
fro_list_data = pd.read_excel(fro_list_path)
geocodes_list_data = pd.read_excel(geocodes_list_path)
location_list_data = pd.read_excel(location_list_path)

In [6]:
fro_worklist_data = pd.read_excel('C://Users//Ani//Downloads//FRO Worklist dump - Mar 1-7.xlsx')

In [4]:
# Set your Azure Maps subscription key
api_key = "dtOo2U8CwwZkR4GAnj06HD3zzU5-yS_dNtKxDkOrtF4"

In [5]:
def get_route(api_key, coordinates):
    url = "https://atlas.microsoft.com/route/directions/json"
    # Ensure coordinates are in [longitude, latitude] format for the query parameter
    coords_str = ":".join([f"{lon},{lat}" for lat, lon in coordinates])
    params = {
        "api-version": "1.0",
        "subscription-key": api_key,
        "query": coords_str,
        'travelMode': 'truck'
    }
    response = requests.get(url, params=params)
    # Check response status before returning JSON to handle possible errors
    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.status_code, "message": response.text}

def fetch_routes_in_batch(api_key, routes):
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        # Submitting batch routing requests
        future_to_route = {executor.submit(get_route, api_key, route): route for route in routes}
        for future in concurrent.futures.as_completed(future_to_route):
            route = future_to_route[future]
            try:
                data = future.result()
            except Exception as exc:
                print(f"{route} generated an exception: {exc}")
            else:
                results.append(data)
    return results

# Example usage with your API key

routes = [
    # Ensure each pair within routes is in [longitude, latitude] order
    [(-122.4783, 37.8199), (-115.1728, 36.1147), (-118.2437, 34.0522)],  # Route 1: Golden Gate Bridge to Las Vegas
    [(-74.0060, 40.7128), (-118.2437, 34.0522)]  # Route 2: New York City to Los Angeles
   # [(-115.1728, 36.1147), (-122.4783, 37.8199)]]  # Route 3: Golden Gate Bridge to Los Angeles
]
batch_results = fetch_routes_in_batch(api_key, routes)

def extract_route_summary(batch_results):
    summaries = []
    for result in batch_results:
        if 'routes' in result:
            for route in result['routes']:
                if 'summary' in route:
                    summary = route['summary']
                    summaries.append({
                        'lengthInMeters': summary.get('lengthInMeters'),
                        'travelTimeInSeconds': summary.get('travelTimeInSeconds')
                    })
    return summaries

route_summaries = extract_route_summary(batch_results)
print(route_summaries)

[{'lengthInMeters': 4609177, 'travelTimeInSeconds': 150830}, {'lengthInMeters': 1353190, 'travelTimeInSeconds': 53854}]


## Main

In [9]:
imports = fro_worklist_data[fro_worklist_data['Traffic Direction'] == 'Import']
exports = fro_worklist_data[fro_worklist_data['Traffic Direction'] == 'Export']

def build_matching_dataframe(imports, exports):


    # Convert 'Appointment Date/Time' to datetime format if not already
    imports['Appointment Date/Time'] = pd.to_datetime(imports['Appointment Date/Time'], errors='coerce')
    exports['Appointment Date/Time'] = pd.to_datetime(exports['Appointment Date/Time'], errors='coerce')

    matching_details_list = []
    
    for _, import_row in imports.iterrows():
        import_date = import_row['Appointment Date/Time'].date()
        
        potential_exports = exports[
            (exports['Source City'] == import_row['Source City']) &
            #(exports['Means of Transport'] == import_row['Means of Transport']) &
            (exports['Equipment Group'] == import_row['Equipment Group']) &
            (exports['Equipment Type'] == import_row['Equipment Type']) &
            exports['Appointment Date/Time'].dt.date.isin([import_date, import_date + timedelta(days=1)])
        ]
        
        for _, export_row in potential_exports.iterrows():
            matching_details_list.append({
                "Source City": import_row['Source City'],
                "Source Coordinates": (import_row['source_longitude'], import_row['source_latitude']),
                "Import Customer City": import_row['Customer City Name'],
                "Import Coordinates": (import_row['destination_longitude'], import_row['destination_latitude']),
                "Export Customer City": export_row['Customer City Name'],
                "Export Coordinates": (export_row['destination_longitude'], export_row['destination_latitude']),
                "Import Appointment Date/Time": import_row['Appointment Date/Time'],
                "Export Appointment Date/Time": export_row['Appointment Date/Time']
            })
    
    return pd.DataFrame(matching_details_list)

# Assuming 'imports' and 'exports' are defined DataFrames
matching_df = build_matching_dataframe(imports, exports)
matching_df.head()


c:\Users\Ani\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\Ani\AppData\Local\Temp\ipykernel_25792\1381183321.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imports['Appointment Date/Time'] = pd.to_datetime(imports['Appointment Date/Time'], errors='coerce')
c:\Users\Ani\anaconda3\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CET identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone

,Source City,Source Coordinates,Import Customer City,Import Coordinates,Export Customer City,Export Coordinates,Import Appointment Date/Time,Export Appointment Date/Time
0,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Pinarolo Po,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:00:00
1,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 07:00:00
2,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 07:30:00
3,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:00:00
4,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:30:00


In [12]:
def preprocess_input_data(input_data):
    unique_routes = []
    for idx, entry in enumerate(input_data):
        source_coords = entry['Source Coordinates']
        import_coords = entry['Import Coordinates']
        export_coords = entry['Export Coordinates']


        # Append route information as dictionaries including descriptive text
        unique_routes.append([source_coords, import_coords, source_coords])
        unique_routes.append([source_coords, export_coords, source_coords])
        unique_routes.append([source_coords, import_coords, export_coords, source_coords])
        unique_routes.append([import_coords, export_coords])

    return unique_routes


routes = preprocess_input_data(matching_df.to_dict(orient='records'))
len(routes)

13028

In [13]:
batch_results = fetch_routes_in_batch(api_key, routes)

def extract_route_summary(batch_results):
    summaries = []
    for result in batch_results:
        if 'routes' in result:
            for route in result['routes']:
                if 'summary' in route:
                    summary = route['summary']
                    summaries.append({
                        'lengthInMeters': summary.get('lengthInMeters'),
                        'travelTimeInSeconds': summary.get('travelTimeInSeconds')
                    })
    return summaries

route_summaries = extract_route_summary(batch_results)
print(route_summaries)

[{'lengthInMeters': 0, 'travelTimeInSeconds': 0}, {'lengthInMeters': 0, 'travelTimeInSeconds': 0}, {'lengthInMeters': 0, 'travelTimeInSeconds': 0}, {'lengthInMeters': 411032, 'travelTimeInSeconds': 18769}, {'lengthInMeters': 0, 'travelTimeInSeconds': 0}, {'lengthInMeters': 411032, 'travelTimeInSeconds': 18769}, {'lengthInMeters': 411032, 'travelTimeInSeconds': 18769}, {'lengthInMeters': 819940, 'travelTimeInSeconds': 38370}, {'lengthInMeters': 0, 'travelTimeInSeconds': 0}, {'lengthInMeters': 819940, 'travelTimeInSeconds': 38370}, {'lengthInMeters': 819940, 'travelTimeInSeconds': 38372}, {'lengthInMeters': 819940, 'travelTimeInSeconds': 38372}, {'lengthInMeters': 819940, 'travelTimeInSeconds': 38372}, {'lengthInMeters': 819940, 'travelTimeInSeconds': 38372}, {'lengthInMeters': 0, 'travelTimeInSeconds': 0}, {'lengthInMeters': 819940, 'travelTimeInSeconds': 38370}, {'lengthInMeters': 0, 'travelTimeInSeconds': 0}, {'lengthInMeters': 819940, 'travelTimeInSeconds': 38372}, {'lengthInMeters':

the code below proves no travel time takes more than a day

In [53]:
count = sum(summary['travelTimeInSeconds'] > 86400 for summary in route_summaries)
count


0

In [45]:
len(route_summaries)

18394

In [14]:
def add_route_summary_columns(matching_df, route_summaries):
    # Initialize columns with default values
    matching_df['Import Time'] = 0
    matching_df['Import Distance'] = 0
    matching_df['Export Time'] = 0
    matching_df['Export Distance'] = 0
    matching_df['Triangulation Time'] = 0
    matching_df['Triangulation Distance'] = 0
    matching_df['Import Export Time'] = 0
    matching_df['Import Export Distance'] = 0
    
    # Iterate through matching_df with step size of 4 in route_summaries
    for idx in range(0, len(matching_df)):
        # Calculate the starting index in route_summaries for the current row
        summary_idx = idx * 4

        # Check if there are enough entries in route_summaries for the current row
        if summary_idx + 3 < len(route_summaries):
            matching_df.at[idx, 'Import Time'] = route_summaries[summary_idx]['travelTimeInSeconds']
            matching_df.at[idx, 'Import Distance'] = route_summaries[summary_idx]['lengthInMeters']
            matching_df.at[idx, 'Export Time'] = route_summaries[summary_idx + 1]['travelTimeInSeconds']
            matching_df.at[idx, 'Export Distance'] = route_summaries[summary_idx + 1]['lengthInMeters']
            matching_df.at[idx, 'Triangulation Time'] = route_summaries[summary_idx + 2]['travelTimeInSeconds']
            matching_df.at[idx, 'Triangulation Distance'] = route_summaries[summary_idx + 2]['lengthInMeters']
            matching_df.at[idx, 'Import Export Time'] = route_summaries[summary_idx + 3]['travelTimeInSeconds']
            matching_df.at[idx, 'Import Export Distance'] = route_summaries[summary_idx + 3]['lengthInMeters']
        else:
            # If not enough entries, print the row number and insert 0s
            print(f"Row {idx + 1} is missing route summaries; inserting default values.")
            # Default values are already set during initialization, so no further action is needed for insertion

    return matching_df


updated_df = add_route_summary_columns(matching_df, route_summaries)

Row 3235 is missing route summaries; inserting default values.
Row 3236 is missing route summaries; inserting default values.
Row 3237 is missing route summaries; inserting default values.
Row 3238 is missing route summaries; inserting default values.
Row 3239 is missing route summaries; inserting default values.
Row 3240 is missing route summaries; inserting default values.
Row 3241 is missing route summaries; inserting default values.
Row 3242 is missing route summaries; inserting default values.
Row 3243 is missing route summaries; inserting default values.
Row 3244 is missing route summaries; inserting default values.
Row 3245 is missing route summaries; inserting default values.
Row 3246 is missing route summaries; inserting default values.
Row 3247 is missing route summaries; inserting default values.
Row 3248 is missing route summaries; inserting default values.
Row 3249 is missing route summaries; inserting default values.
Row 3250 is missing route summaries; inserting default 

In [15]:
updated_df

,Source City,Source Coordinates,Import Customer City,Import Coordinates,Export Customer City,Export Coordinates,Import Appointment Date/Time,Export Appointment Date/Time,Import Time,Import Distance,Export Time,Export Distance,Triangulation Time,Triangulation Distance,Import Export Time,Import Export Distance
0,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Pinarolo Po,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:00:00,0,0,0,0,0,0,18769,411032
1,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 07:00:00,0,0,18769,411032,18769,411032,38370,819940
2,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 07:30:00,0,0,38370,819940,38372,819940,38372,819940
3,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:00:00,38372,819940,38372,819940,0,0,38370,819940
4,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:30:00,0,0,38372,819940,38370,819940,18769,411032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3252,Piacenza,"(8.45306, 44.26198)",Bologna,"(11.27394, 44.52692)",Bazzano,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 10:00:00,0,0,0,0,0,0,0,0
3253,Piacenza,"(8.45306, 44.26198)",Bologna,"(11.27394, 44.52692)",Crespiatica,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 08:30:00,0,0,0,0,0,0,0,0
3254,Piacenza,"(8.45306, 44.26198)",Bologna,"(11.27394, 44.52692)",VALSAMOGGIA,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 08:30:00,0,0,0,0,0,0,0,0
3255,Piacenza,"(8.45306, 44.26198)",Bologna,"(11.27394, 44.52692)",Mirandola,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 09:00:00,0,0,0,0,0,0,0,0


however, we see here that there now times that suddenly exceed a day

In [16]:
filtered_df = updated_df[(updated_df['Import Time'] > 86400) | (updated_df['Export Time'] > 86400) | (updated_df['Triangulation Time'] > 86400) | (updated_df['Import Export Time'] > 86400)]
filtered_df

,Source City,Source Coordinates,Import Customer City,Import Coordinates,Export Customer City,Export Coordinates,Import Appointment Date/Time,Export Appointment Date/Time,Import Time,Import Distance,Export Time,Export Distance,Triangulation Time,Triangulation Distance,Import Export Time,Import Export Distance


In [17]:
def create_triangulation_potentials_dfs(df):
    # Ensure "Import Export Time" is treated as numeric seconds
    df['Import Export Time'] = pd.to_numeric(df['Import Export Time'], errors='coerce')

    # Initialize empty DataFrames for triangulations and potentials
    triangulations = pd.DataFrame(columns=df.columns)
    potentials = pd.DataFrame(columns=df.columns)
    
    for idx, row in df.iterrows():
        # Calculate time difference between Export and Import Appointments in seconds
        time_diff = (pd.to_datetime(row['Export Appointment Date/Time']) - pd.to_datetime(row['Import Appointment Date/Time'])).total_seconds()
        
        # Check if the condition for triangulation is met
        if time_diff > row['Import Export Time']:
            triangulations = triangulations.append(row, ignore_index=True)
        else:
            potentials = potentials.append(row, ignore_index=True)
    
    return triangulations, potentials

# Example usage
# Ensure that updated_df is defined and contains the correct columns before running this
triangulations_df, potentials_df = create_triangulation_potentials_dfs(updated_df)

C:\Users\Ani\AppData\Local\Temp\ipykernel_25792\771432245.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  potentials = potentials.append(row, ignore_index=True)
C:\Users\Ani\AppData\Local\Temp\ipykernel_25792\771432245.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  triangulations = triangulations.append(row, ignore_index=True)


In [18]:
triangulations_df

,Source City,Source Coordinates,Import Customer City,Import Coordinates,Export Customer City,Export Coordinates,Import Appointment Date/Time,Export Appointment Date/Time,Import Time,Import Distance,Export Time,Export Distance,Triangulation Time,Triangulation Distance,Import Export Time,Import Export Distance
0,Genoa Vado Ligure,"(8.45306, 44.26198)",Stradella,"(9.28304, 45.09155)",RIVALTA SCRIVIA INTERPORTO,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 08:30:00,16532,309411,7742,155761,10337,211248,0,0
1,Genoa Vado Ligure,"(8.45306, 44.26198)",Stradella,"(9.28304, 45.09155)",Desio,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 09:00:00,16532,309411,16532,309411,16532,309411,0,0
2,La Spezia,"(9.85774, 44.1035)",Pontenure,"(9.78486, 45.00777)",Desenzano del Garda,"(9.85774, 44.1035)",2024-04-03 07:00:00,2024-04-03 14:00:00,15134,314839,7358,160194,23797,449650,690,4109
3,La Spezia,"(9.85774, 44.1035)",Pontenure,"(9.78486, 45.00777)",Marlia,"(9.8468573, 44.1112051)",2024-04-03 07:00:00,2024-04-03 08:00:00,15516,317667,7469,160355,15134,314839,690,4109
4,La Spezia,"(9.85774, 44.1035)",Pontenure,"(9.78486, 45.00777)",Marlia,"(9.8468573, 44.1112051)",2024-04-03 07:00:00,2024-04-03 09:00:00,7469,160355,15516,317667,15134,314839,690,4109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,Piacenza,"(8.45306, 44.26198)",Bologna,"(11.27394, 44.52692)",Bazzano,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 09:00:00,0,0,0,0,0,0,0,0
749,Piacenza,"(8.45306, 44.26198)",Bologna,"(11.27394, 44.52692)",Bazzano,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 10:00:00,0,0,0,0,0,0,0,0
750,Piacenza,"(8.45306, 44.26198)",Bologna,"(11.27394, 44.52692)",Crespiatica,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 08:30:00,0,0,0,0,0,0,0,0
751,Piacenza,"(8.45306, 44.26198)",Bologna,"(11.27394, 44.52692)",VALSAMOGGIA,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 08:30:00,0,0,0,0,0,0,0,0


In [19]:
potentials_df

,Source City,Source Coordinates,Import Customer City,Import Coordinates,Export Customer City,Export Coordinates,Import Appointment Date/Time,Export Appointment Date/Time,Import Time,Import Distance,Export Time,Export Distance,Triangulation Time,Triangulation Distance,Import Export Time,Import Export Distance
0,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Pinarolo Po,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:00:00,0,0,0,0,0,0,18769,411032
1,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 07:00:00,0,0,18769,411032,18769,411032,38370,819940
2,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 07:30:00,0,0,38370,819940,38372,819940,38372,819940
3,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:00:00,38372,819940,38372,819940,0,0,38370,819940
4,Genoa Vado Ligure,"(8.45306, 44.26198)",Galliera Veneta,"(11.84088, 45.66225)",Verzuolo,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:30:00,0,0,38372,819940,38370,819940,18769,411032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2499,Piacenza,"(8.45306, 44.26198)",Borgo San Giovanni,"(9.42238, 45.27432)",Pegognaga,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:00:00,0,0,0,0,0,0,0,0
2500,Piacenza,"(8.45306, 44.26198)",Borgo San Giovanni,"(9.42238, 45.27432)",Correggio,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 06:00:00,0,0,0,0,0,0,0,0
2501,Piacenza,"(8.45306, 44.26198)",Borgo San Giovanni,"(9.42238, 45.27432)",Finale Emilia,"(8.45306, 44.26198)",2024-01-03 08:00:00,2024-01-03 08:00:00,0,0,0,0,0,0,0,0
2502,Piacenza,"(8.45306, 44.26198)",Bologna,"(11.27394, 44.52692)",Sassuolo; Casalgrande,"(8.45306, 44.26198)",2024-06-03 08:00:00,2024-06-03 08:00:00,0,0,0,0,0,0,0,0
